In [12]:
import os
from tqdm import tqdm
import numpy as np
from Bio import SeqIO

In [13]:
#saves sequences in fasta file as array
def read_fasta(file_path): 
    sequences = []
    with open(file_path, 'r') as file:
        sequence = ''
        for line in file:
            if line.startswith('>'):
                if sequence:  
                    sequences.append(sequence)
                    sequence = ''
            else:
                sequence += line.strip()  
                
        if sequence: 
            sequences.append(sequence)
            
    return sequences

In [14]:
file_path = "/Volumes/Extreme SSD/HP/Scripts/read_1.fasta"
sequences = read_fasta(file_path)
names = []

In [15]:
#Saves names of sequences as array
with open(file_path, "r") as fasta_file:
    for record in tqdm(SeqIO.parse(fasta_file, "fasta")):
        names.append(record.id)

177397it [00:00, 632271.37it/s]


In [16]:
#Generate palindromes for cap1 and cap2 tags
def reverse_complement_cap2(reverse_triplet):
    complement = {'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C'}
    return ''.join(complement[n] for n in reverse_triplet)

def generate_palindromes_cap2():
    nucleotides = ['A', 'T', 'C', 'G']
    #no_a = ['T', 'C', 'G']
    palindromes_cap2 = []

    for n1 in nucleotides:
        for n2 in nucleotides:
            for n3 in nucleotides:
                triplet = n1 + n2 + n3
                reverse_triplet = n3+n2+n1
                palindrome = triplet + reverse_complement_cap2(reverse_triplet)
                if (n1!=n2 or n2!=n3 or n3!=n1):
                    palindromes_cap2.append(palindrome)

    return palindromes_cap2

def reverse_complement_cap1(reverse_doublet):
    complement = {'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C'}
    return ''.join(complement[n] for n in reverse_doublet)

def generate_palindromes_cap1():
    nucleotides = ['A', 'T', 'C', 'G']
    #no_a = ['T', 'G', 'C']
    palindromes_cap1 = []

    for n1 in nucleotides:
        for n2 in nucleotides:
            doublet = n1 + n2
            reverse_doublet = n2+n1
            palindrome = doublet + reverse_complement_cap1(reverse_doublet)
            if n1!=n2:
                palindromes_cap1.append(palindrome)

    return palindromes_cap1
    
# Generate the palindromes
palindromes_cap2 = generate_palindromes_cap2()
palindromes_cap1 = generate_palindromes_cap1()

In [17]:
def match_palindrome_cap2(sequence, palindromes, result_array0, result_array1, result_array0_name, result_array1_name, name):
    first_three_chars = sequence[:3]
    chars_5_to_7 = sequence[4:7]
    first_6 = sequence[:6]
    for s in palindromes:
        if first_three_chars == s[:3] and chars_5_to_7 == s[3:6]:
            result_array1.append(sequence)
            result_array1_name.append(name)
            break
        elif first_6 == s[:6]:
            result_array0.append(sequence)
            result_array0_name.append(name)
            break

        

def match_palindrome_cap1(sequence, palindromes, result_array0, result_array1, result_array0_name, result_array1_name, name):
    # Extract the first three characters and 5-7 characters of the string to check
    first_two_chars = sequence[:2]
    chars_4_to_5 = sequence[3:5]
    first_4 = sequence[:4]
    # Loop through the list of strings to find a match
    for s in palindromes:
        if first_two_chars == s[:2] and chars_4_to_5 == s[2:4]:
            result_array1.append(sequence)
            result_array1_name.append(name)
            break
        elif first_4 == s[:4]:
            result_array0.append(sequence)
            result_array0_name.append(name)
            break


    

result_array_cap2_0 = []
result_array_cap2_1 = []
cap2_0_name = []
cap2_1_name = []
for i in tqdm(range(len(sequences))):
    match_palindrome_cap2(sequences[i], palindromes_cap2, result_array_cap2_0, result_array_cap2_1, cap2_0_name, cap2_1_name, names[i])

result_array_cap1_0 = []
result_array_cap1_1 = []
cap1_0_name = []
cap1_1_name = []

for i in tqdm(range(len(sequences))):
    match_palindrome_cap1(sequences[i], palindromes_cap1, result_array_cap1_0, result_array_cap1_1, cap1_0_name, cap1_1_name, names[i])


100%|████████████████████████████████████████████████████████████████████████████| 177397/177397 [00:00<00:00, 1132015.71it/s]


In [18]:
import gc

del palindromes_cap1
del palindromes_cap2
del sequences
gc.collect()   

0

In [22]:
#write cap1 arrays to fasta file with 40 bp for alignment step
def write_fasta_cap1(sequence0, sequence1, name0, name1, file_name):
    with open(file_name, 'w') as file:
        for i, sequence in tqdm(enumerate(sequence0)):
            if len(sequence)>=42:
                file.write(f">Sequence{i+1}_{name0[i]}_{sequence[:2]}\n")
                file.write(sequence[2:42] + "\n") 
        for i, sequence in tqdm(enumerate(sequence1)):
            if len(sequence)>=43:
                file.write(f">Sequence{i+1}_{name1[i]}_{sequence[:3]}\n")
                file.write(sequence[3:43] + "\n")

In [23]:
#write cap2 arrays to fasta file with 40 bp for alignment step
def write_fasta_cap2(sequence0, sequence1, name0, name1, file_name):
    with open(file_name, 'w') as file:
        for i, sequence in tqdm(enumerate(sequence0)):
            if len(sequence) >= 43:
                file.write(f">Sequence{i+1}_{name0[i]}_{sequence[:3]}\n")
                file.write(sequence[3:43] + "\n")
        for i, sequence in tqdm(enumerate(sequence1)):
            if len(sequence) >= 44:
                file.write(f">Sequence{i+1}_{name1[i]}_{sequence[:4]}\n")
                file.write(sequence[4:44] + "\n")

In [24]:
write_fasta_cap1(result_array_cap1_0, result_array_cap1_1, cap1_0_name, cap1_1_name, "/Volumes/Extreme SSD/HP/cap1_m6am_test.fasta")
write_fasta_cap2(result_array_cap2_0, result_array_cap2_1, cap2_0_name, cap2_1_name, "/Volumes/Extreme SSD/HP/cap2_m6am_test.fasta")

5453it [00:00, 302826.01it/s]
6375it [00:00, 1576759.52it/s]
1060it [00:00, 1693054.93it/s]
2035it [00:00, 1547390.98it/s]
